# WebComponent Reference Guide

In [ ]:
import param
import panel as pn

js_files = {
    'webcomponents': "https://unpkg.com/@webcomponents/webcomponentsjs@2.2.7/webcomponents-loader.js",
    'model_viewer': "https://unpkg.com/@google/model-viewer/dist/model-viewer-legacy.js",
    'resize': "https://unpkg.com/resize-observer-polyfill@1.5.1/dist/ResizeObserver.js",
    'echarts': "https://cdn.bootcss.com/echarts/3.7.2/echarts.min.js",
}

pn.extension(js_files=js_files)

pn.config.sizing_mode="stretch_width"

WebComponent
============

Use the WebComponent to quickly **plugin** webcomponents and/ or Javascript libraries not already
supported in Panel.

You can use it by instantiating an instance or inheriting from it.

Parameters
---------------

- `attributes_to_watch`: A dictionary of (`html_attribute`, `python_parameter`) names
    - The value of `python_parameter`  will be used to set the `html_attribute` on construction. 
    - The value of `python_parameter` and `html_attribute` will be kept in sync (*two way binding*).
    - The value of `html_attribute` may not be None
    - The value of `python_parameter` can be None.
- `properties_to_watch`: A dictionary of (`js_property`, `python_parameter`) names
    - The value of `python_parameter`  will be used to set the `js_property` on construction. 
    - The value of `python_parameter` and `js_property` will be kept in sync (*two way binding*).
    - The value of `js_property` may not be None
    - The value of `python_parameter` can be None.
    - You can specify a nested `js_property` like `textInput.value` as key.
- `events_to_watch`: A Dictionary of (`js_event`, `python_parameter_to_inc`) names.
    - The `js_event` will be watched on the JavaScript side. When fired the javascript code will 
        - Increment the `python_parameter_to_inc` with `+1` if specified.
        - check whether any `js_property` key in `properties_to_watch` has changed and if yes then it will be synced to the associated `python_parameter` value.
- The `column_data_source` can be used to efficiently transfer columnar data to the javascript side.
    - The `column_data_source_orient` is used to specify how the data should be input to the `column_data_source_load_function` below.
         - For now `dict` and `records` are supported.
         - See the `orient` parameter of the [pandas.to_dict](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_dict.html) function for more info.
- The `column_data_source_load_function` specifies the name of the js function on the that will load the data.

Other
-------

- If the JavaScript WebComponent contains an `after_layout` function this is used to
resize the JS WebComponent. An example is the ECharts web component.

- You will propably have to experiment a bit in order to determine which
javascript files to import and what combination of attributes, properties, events and/ or
parameters to watch.
___

## Introduction

Web Components powers the modern web. For an introduction and overview see [Web Components: The Secret Ingredient Helping Power The Web](https://www.youtube.com/watch?v=YBwgkr_Sbx0).

You can find web components to use in your Panel Project at 

- [webcomponent.org](https://www.webcomponents.org/)
- [npm](https://www.npmjs.com/), 

Please note that picking and using a web component library can be **challenging** as the web component standard and packages are rapidly evolving. You might have to do some experimentation before it works for you.

## Google Model Viewer

Google has developed a `model-viewer` web component for interactively viewing very large and detailed 3D models. For an introduction see [modelviewer.dev](https://modelviewer.dev/)

In [ ]:
html = """
<model-viewer src="https://modelviewer.dev/shared-assets/models/Astronaut.glb" alt="A 3D model of an astronaut"
auto-rotate camera-controls style="height:100%;width:100%;">
</model-viewer>
"""

pn.pane.WebComponent(html=html, height=300)

Try dragging the astronaut around. Please note that we haveve specified a **fixed height** for the `model-viewer`. This is important to do.

### Advanced Version

Please that this example **might take 15 secs to 2 minutes to load** depending on you internet connection.

In [ ]:
astronaut_src = "https://modelviewer.dev/shared-assets/models/Astronaut.glb"
flight_helmet_src="https://modelviewer.dev/shared-assets/models/glTF-Sample-Models/2.0/FlightHelmet/glTF/FlightHelmet.gltf"

class ModelViewer(pn.pane.WebComponent):
    """A Wired ModelViewer"""
    html = param.String(html)
    attributes_to_watch= param.Dict({"src": "src"})
    properties_to_watch= param.Dict({"exposure": "exposure"})

    src = param.String(astronaut_src)
    exposure = param.Number(1.0, bounds=(0, 2))


model_viewer = ModelViewer(height=600, src=flight_helmet_src)

pn.Row(
    model_viewer, pn.Param(model_viewer, parameters=["exposure"]),
)

To get inspiration for a more interactive version see [model-viewer tester](https://modelviewer.dev/examples/tester.html).

## ECharts/ PyeChart

[ECharts](https://www.echartsjs.com/en/index.html) is an open-sourced JavaScript
visualization tool, which can run fluently on PC and mobile devices.
It is compatible with most modern Web Browsers. Its also an **Apache incubator project**.

[Pyecharts](https://pyecharts.org/#/en-us/) is a Python api for using ECharts in Python
including Standalone, Flask, Django and Jupyter Notebooks.

Below we develop an `EChart` pane capable of showing Echarts dicts and Pyecharts objects.

In [ ]:
ECHARTS_HTML = """
<div class="echart" style="width:100%;height:100%;"></div>
<script type="text/javascript">
    var myScript = document.currentScript;
    var myDiv = myScript.parentElement.firstElementChild;
    var myChart = echarts.init(myDiv);
    myDiv.eChart = myChart;
    Object.defineProperty(myDiv, 'option', {
       get: function() { return null; },
       set: function(val) { this.eChart.setOption(val); this.eChart.resize();}
    });
    myDiv.after_layout = myChart.resize; // Resizes the chart after layout of parent element
</script>
"""

class EChart(pn.pane.WebComponent):
    html = param.String(ECHARTS_HTML)
    properties_to_watch = param.Dict({"option": "option"})

    echart = param.Parameter()
    option = param.Dict()

    def __init__(self, **params):
        if "echart" in params:
            params["option"] = self._to_echart_dict(params["echart"])
        super().__init__(**params)

    @classmethod
    def _to_echart_dict(cls, echart):
        if isinstance(echart, dict):
            return echart
        if 'pyecharts' in sys.modules:
            import pyecharts
            if isinstance(echart, pyecharts.charts.chart.Chart):
                return json.loads(echart.dump_options())

        return {}

    @param.depends("echart", watch=True)
    def update(self):
        self.option = self._to_echart_dict(self.echart)

In [ ]:
ECHART_DICT = {
    "title": {"text": "ECharts entry example"},
    "tooltip": {},
    "legend": {"data": ["Sales"]},
    "xAxis": {"data": ["shirt", "cardign", "chiffon shirt", "pants", "heels", "socks"]},
    "yAxis": {},
    "series": [{"name": "Sales", "type": "bar", "data": [5, 20, 36, 10, 10, 20]}],
}

app = pn.Column(
    EChart(echart=ECHART_DICT, height=500),
)

app

## WiredJS

[Wiredjs](https://wiredjs.com/) is a set of common UI elements with a hand-drawn, sketchy look. These can be used for wireframes, mockups, or just the fun hand-drawn look.

For a demo of all available components see the [WiredJS ShowCase](https://wiredjs.com/showcase.html)

In [ ]:
js = """
<script src="https://unpkg.com/@webcomponents/webcomponentsjs@2.2.7/webcomponents-loader.js"></script>
<script src="https://wiredjs.com/dist/showcase.min.js"></script>
"""

ELEVATION_DEFAULT = 0
ELEVATION_BOUNDS = (0, 10)

class WiredBase(pn.pane.WebComponent):
    """Inherit from this class"""

    def __init__(self, **params):
        if not self.param.attributes_to_watch.default:
            self.param.attributes_to_watch.default = {}
        self.attributes_to_watch["disabled"] = "disabled"

        super().__init__(**params)

    def _child_parameters(self):
        parameters = super()._child_parameters()
        parameters.add("disabled")
        return parameters


class Button(WiredBase):
    """A Wired RadioButton

    - You can set the `text` shown via the `name` parameter.
    """

    html = param.String("<wired-button ></wired-button>")
    attributes_to_watch = param.Dict({"elevation": "elevation"})
    events_to_watch = param.Dict(default={"click": "clicks"})
    parameters_to_watch = param.List(["name"])

    clicks = param.Integer()
    elevation = param.Integer(ELEVATION_DEFAULT, bounds=ELEVATION_BOUNDS)
    
    def __init__(self, **params):
        if "height" not in params:
            params["height"]=40
        super().__init__(**params)

    def _get_html_from_parameters_to_watch(self, **params) -> str:
        return f"<wired-button>{params['name']}</wired-button>"

class Checkbox(WiredBase):
    html = param.String("<wired-checkbox></wired-checkbox>")
    properties_to_watch = param.Dict({"checked": "value"})
    parameters_to_watch = param.List(["name"])

    value = param.Boolean()
    
    def __init__(self, **params):
        if "height" not in params:
            params["height"]=40
        
        super().__init__(**params)

    def _get_html_from_parameters_to_watch(self, **params) -> str:
        return f"<wired-checkbox>{params['name']}</wired-checkbox>"

In [ ]:
js_pane = pn.pane.HTML(js)

button = Button(name="Click me")
check_box = Checkbox(name="Check me")

pn.Column(
    js, 
    pn.Row(
        pn.Column("## Button", button, pn.Param(button, parameters=["name", "clicks", "elevation"])), 
        pn.layout.VSpacer(width=5, background="lightgrey"),
        pn.Column("## CheckBox", check_box, pn.Param(check_box, parameters=["name", "value"])),
    )
)

## Web Component Libraries

You can find web components at

- [webcomponents.org](https://www.webcomponents.org/)
- [npm](https://www.npmjs.com/)

Some example libraries are

- [Material](https://github.com/material-components/material-components-web-components#readme) (Work in Progress)
- [Vaadin](https://vaadin.com/components)
- [Smart Elements](https://www.webcomponents.org/element/@smarthtmlelements/smart-bootstrap)

### Prebuilt libraries from unpkg.com

If you find a web component library on [npm](https://npmjs.com) you can find the corresponding precombiled library on [unpkg](https://unpkg.com) for use with Panel.

For example if you find the `wired-button` at [https://www.npmjs.com/package/wired-button](https://www.npmjs.com/package/wired-button) then you can browse the precompiled files at [https://www.unpkg.com/browse/wired-button/](https://www.unpkg.com/browse/wired-button/) to locate the relevant precombiled file at [https://www.unpkg.com/wired-button@2.0.0/lib/wired-button.js](https://www.unpkg.com/wired-button@2.0.0/lib/wired-button.js).

## References

- [Build an app with WebComponents in 9 minutes](https://www.youtube.com/watch?v=mTNdTcwK3MM)
- [How to use Web Components in a JavaScript project](https://www.youtube.com/watch?v=88Sa-SlHRxk&t=63s)

## Credits

- [Marc Skov Maddsen](datamodelsanalytics.com), [awesome-panel.org](https://awesome-panel.org)